# **TTE_v1: Convert R Codes into Python**
## **1. Setup**

In [74]:
import os
import tempfile

# Define estimands as variables
trial_pp_estimand = "PP"   # Per-protocol
trial_itt_estimand = "ITT"  # Intention-to-treat

# Create directories to save files
trial_pp_dir = os.path.join(tempfile.gettempdir(), "trial_pp")
trial_itt_dir = os.path.join(tempfile.gettempdir(), "trial_itt")

os.makedirs(trial_pp_dir, exist_ok=True)
os.makedirs(trial_itt_dir, exist_ok=True)

print(f"PP Trial Directory: {trial_pp_dir}")
print(f"ITT Trial Directory: {trial_itt_dir}")

PP Trial Directory: /var/folders/j1/98lnr0rj7px6w8q4b_d6mlmm0000gn/T/trial_pp
ITT Trial Directory: /var/folders/j1/98lnr0rj7px6w8q4b_d6mlmm0000gn/T/trial_itt


## **2. Data Preparation**

In [75]:
import pandas as pd

# Load the dataset 
data_censored = pd.read_csv("../data/data_censored.csv")

# Display the first few rows
print(data_censored.head())

# Define a function to structure the trial data
class TrialSequence:
    def __init__(self, estimand, data, id_col, period_col, treatment_col, outcome_col, eligible_col):
        self.estimand = estimand
        self.data = data
        self.id_col = id_col
        self.period_col = period_col
        self.treatment_col = treatment_col
        self.outcome_col = outcome_col
        self.eligible_col = eligible_col

    def summary(self):
        return {
            "Estimand": self.estimand,
            "Observations": self.data.shape[0],
            "Patients": self.data[self.id_col].nunique(),
            "Columns": self.data.columns.tolist(),
            "Sample Data": self.data.head()
        }

# Create the Per-Protocol (PP) trial sequence
trial_pp = TrialSequence(
    estimand="PP",
    data=data_censored,
    id_col="id",
    period_col="period",
    treatment_col="treatment",
    outcome_col="outcome",
    eligible_col="eligible"
)

# Create the Intention-to-Treat (ITT) trial sequence
trial_itt = TrialSequence(
    estimand="ITT",
    data=data_censored,
    id_col="id",
    period_col="period",
    treatment_col="treatment",
    outcome_col="outcome",
    eligible_col="eligible"
)

# Print a summary of the ITT trial
print(trial_itt.summary())


   id  period  treatment  x1        x2  x3        x4  age     age_s  outcome  \
0   1       0          1   1  1.146148   0  0.734203   36  0.083333        0   
1   1       1          1   1  0.002200   0  0.734203   37  0.166667        0   
2   1       2          1   0 -0.481762   0  0.734203   38  0.250000        0   
3   1       3          1   0  0.007872   0  0.734203   39  0.333333        0   
4   1       4          1   1  0.216054   0  0.734203   40  0.416667        0   

   censored  eligible  
0         0         1  
1         0         0  
2         0         0  
3         0         0  
4         0         0  
{'Estimand': 'ITT', 'Observations': 725, 'Patients': 89, 'Columns': ['id', 'period', 'treatment', 'x1', 'x2', 'x3', 'x4', 'age', 'age_s', 'outcome', 'censored', 'eligible'], 'Sample Data':    id  period  treatment  x1        x2  x3        x4  age     age_s  outcome  \
0   1       0          1   1  1.146148   0  0.734203   36  0.083333        0   
1   1       1          1  

## **3. Weight models and censoring**

To adjust for the effects of informative censoring, inverse probability of censoring weights (IPCW) can be applied. To estimate these weights, we construct time-to-(censoring) event models. Two sets of models are fit for the two censoring mechanisms which may apply: censoring due to deviation from assigned treatment and other informative censoring.

### **3.1 Censoring due to treatment switching**

We specify model formulas to be used for calculating the probability of receiving treatment in the current period. Separate models are fitted for patients who had treatment = 1 and those who had treatment = 0 in the previous period. Stabilized weights are used by fitting numerator and denominator models.

There are optional arguments to specify columns which can include/exclude observations from the treatment models. These are used in case it is not possible for a patient to deviate from a certain treatment assignment in that period.



In [76]:
import statsmodels.api as sm
import pickle
import os

def fit_logistic_regression(data, numerator, denominator, save_path):
    """
    Fits a logistic regression model using statsmodels.

    :param data: Pandas DataFrame containing the dataset.
    :param numerator: Formula string for the numerator (predictor variables).
    :param denominator: Formula string for the denominator (predictor variables).
    :param save_path: Path to save the trained model.
    :return: Fitted logistic regression model.
    """
    data = data.dropna(subset=numerator + denominator)  # Remove missing values

    # Fit the numerator model (logistic regression)
    X_num = sm.add_constant(data[numerator])  # Add intercept
    y_num = data['eligible']  # Assuming eligibility is the binary outcome variable
    model_num = sm.Logit(y_num, X_num).fit(disp=False)

    # Fit the denominator model (logistic regression)
    X_denom = sm.add_constant(data[denominator])
    model_denom = sm.Logit(y_num, X_denom).fit(disp=False)

    # Save models
    with open(save_path, "wb") as f:
        pickle.dump({"numerator_model": model_num, "denominator_model": model_denom}, f)

    return model_num, model_denom

# Define the predictor variables
numerator_vars = ["age"]
denominator_vars = ["age", "x1", "x3"]

# Define file path for saving the model
switch_model_path = os.path.join(trial_pp_dir, "switch_models.pkl")

# Fit the switch weight model and save it
trial_pp.switch_weights = fit_logistic_regression(
    trial_pp.data,
    numerator=numerator_vars,
    denominator=denominator_vars,
    save_path=switch_model_path
)

# Print model summary
print(trial_pp.switch_weights[0].summary())  # Summary of the numerator model


                           Logit Regression Results                           
Dep. Variable:               eligible   No. Observations:                  725
Model:                          Logit   Df Residuals:                      723
Method:                           MLE   Df Model:                            1
Date:                Sun, 09 Mar 2025   Pseudo R-squ.:                 0.09696
Time:                        16:39:33   Log-Likelihood:                -356.57
converged:                       True   LL-Null:                       -394.86
Covariance Type:            nonrobust   LLR p-value:                 2.120e-18
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.9822      0.383      5.180      0.000       1.232       2.732
age           -0.0692      0.008     -8.152      0.000      -0.086      -0.053


### **3.2 Other informative censoring**

In [77]:
import statsmodels.api as sm
import pickle
import os

def fit_censor_weight_model(data, censor_event, numerator, denominator, save_path):
    """
    Fits a logistic regression model for censoring weights.

    :param data: Pandas DataFrame containing the dataset.
    :param censor_event: Column name indicating whether an event was censored.
    :param numerator: List of predictor variables for the numerator model.
    :param denominator: List of predictor variables for the denominator model.
    :param save_path: Path to save the trained models.
    :return: Tuple of fitted logistic regression models (numerator, denominator).
    """
    data = data.dropna(subset=numerator + denominator + [censor_event])  # Remove missing values

    # Fit the numerator model (logistic regression)
    X_num = sm.add_constant(data[numerator])  # Add intercept
    y_num = data[censor_event]  # Assuming 'censored' is a binary column (1 = censored, 0 = not censored)
    model_num = sm.Logit(y_num, X_num).fit(disp=False)

    # Fit the denominator model (logistic regression)
    X_denom = sm.add_constant(data[denominator])
    model_denom = sm.Logit(y_num, X_denom).fit(disp=False)

    # Save models
    with open(save_path, "wb") as f:
        pickle.dump({"numerator_model": model_num, "denominator_model": model_denom}, f)

    return model_num, model_denom

# Define the predictor variables
numerator_vars = ["x2"]
denominator_vars = ["x2", "x1"]

# Define file path for saving the model
censor_model_path = os.path.join(trial_pp_dir, "censor_models.pkl")

# Fit the censor weight model and save it
trial_pp.censor_weights = fit_censor_weight_model(
    trial_pp.data,
    censor_event="censored",
    numerator=numerator_vars,
    denominator=denominator_vars,
    save_path=censor_model_path
)

# Print model summary
print(trial_pp.censor_weights[0].summary())  # Summary of the numerator model


                           Logit Regression Results                           
Dep. Variable:               censored   No. Observations:                  725
Model:                          Logit   Df Residuals:                      723
Method:                           MLE   Df Model:                            1
Date:                Sun, 09 Mar 2025   Pseudo R-squ.:                 0.02676
Time:                        16:39:36   Log-Likelihood:                -196.70
converged:                       True   LL-Null:                       -202.11
Covariance Type:            nonrobust   LLR p-value:                  0.001007
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.4481      0.141    -17.415      0.000      -2.724      -2.173
x2             0.4486      0.137      3.278      0.001       0.180       0.717


In [78]:
import statsmodels.api as sm
import pickle
import os

def fit_censor_weight_model(data, censor_event, numerator, denominator, save_path, pool_models="numerator"):
    """
    Fits a logistic regression model for censoring weights.

    :param data: Pandas DataFrame containing the dataset.
    :param censor_event: Column name indicating whether an event was censored.
    :param numerator: List of predictor variables for the numerator model.
    :param denominator: List of predictor variables for the denominator model.
    :param save_path: Path to save the trained models.
    :param pool_models: Strategy for model pooling ("numerator", "denominator", or "none").
    :return: Fitted logistic regression model(s).
    """
    data = data.dropna(subset=numerator + denominator + [censor_event])  # Remove missing values

    # Fit the numerator model (logistic regression)
    X_num = sm.add_constant(data[numerator])  # Add intercept
    y_num = data[censor_event]  # Assuming 'censored' is a binary column (1 = censored, 0 = not censored)
    model_num = sm.Logit(y_num, X_num).fit(disp=False)

    # Fit the denominator model if required
    if pool_models != "numerator":
        X_denom = sm.add_constant(data[denominator])
        model_denom = sm.Logit(y_num, X_denom).fit(disp=False)
    else:
        model_denom = None  # Only use the numerator model

    # Save models
    with open(save_path, "wb") as f:
        pickle.dump({"numerator_model": model_num, "denominator_model": model_denom}, f)

    return model_num if pool_models == "numerator" else (model_num, model_denom)

# Define the predictor variables
numerator_vars = ["x2"]
denominator_vars = ["x2", "x1"]

# Define file path for saving the model
censor_model_path = os.path.join(trial_itt_dir, "censor_models.pkl")

# Fit the censor weight model and save it
trial_itt.censor_weights = fit_censor_weight_model(
    trial_itt.data,
    censor_event="censored",
    numerator=numerator_vars,
    denominator=denominator_vars,
    save_path=censor_model_path,
    pool_models="numerator"  # Match R code behavior
)

# Print model summary
print(trial_itt.censor_weights.summary())  # Summary of the numerator model


                           Logit Regression Results                           
Dep. Variable:               censored   No. Observations:                  725
Model:                          Logit   Df Residuals:                      723
Method:                           MLE   Df Model:                            1
Date:                Sun, 09 Mar 2025   Pseudo R-squ.:                 0.02676
Time:                        16:39:39   Log-Likelihood:                -196.70
converged:                       True   LL-Null:                       -202.11
Covariance Type:            nonrobust   LLR p-value:                  0.001007
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.4481      0.141    -17.415      0.000      -2.724      -2.173
x2             0.4486      0.137      3.278      0.001       0.180       0.717


## **4. Calculate Weights**

Next we need to fit the individual models and combine them into weights. This is done with calculate_weights().


In [79]:
class TrialSequence:
    def __init__(self, estimand, data, id_col, period_col, treatment_col, outcome_col, eligible_col):
        self.estimand = estimand
        self.data = data
        self.id_col = id_col
        self.period_col = period_col
        self.treatment_col = treatment_col
        self.outcome_col = outcome_col
        self.eligible_col = eligible_col
        self.weights = None

    def calculate_weights(self):
        """
        Placeholder function for calculating weights.
        This should be replaced with actual weight calculation logic.
        """
        self.weights = f"Weights calculated for {self.estimand} trial"
        return self

    def show_weight_models(self):
        """
        Placeholder function to display weight models.
        """
        if self.weights:
            print(f"Weight models for {self.estimand} trial:", self.weights)
        else:
            print(f"No weights calculated for {self.estimand} trial.")

In [80]:
import os
import tempfile
import pandas as pd
import statsmodels.api as sm
import pickle

class TrialSequence:
    def __init__(self, estimand, data, id_col, period_col, treatment_col, outcome_col, eligible_col):
        self.estimand = estimand
        self.data = data
        self.id_col = id_col
        self.period_col = period_col
        self.treatment_col = treatment_col
        self.outcome_col = outcome_col
        self.eligible_col = eligible_col
        self.switch_weights = None
        self.censor_weights = None
    
    def fit_logistic_regression(self, data, predictor_vars, outcome_var):
        """Fits a logistic regression model."""
        X = sm.add_constant(data[predictor_vars])
        y = data[outcome_var]
        model = sm.Logit(y, X).fit(disp=False)
        return model
    
    def set_switch_weight_model(self, numerator, denominator, save_path):
        """Fits and saves the switch weight model."""
        self.switch_weights = {
            "numerator": self.fit_logistic_regression(self.data, numerator, self.eligible_col),
            "denominator": self.fit_logistic_regression(self.data, denominator, self.eligible_col)
        }
        with open(save_path, "wb") as f:
            pickle.dump(self.switch_weights, f)
    
    def set_censor_weight_model(self, censor_event, numerator, denominator, save_path):
        """Fits and saves the censor weight model."""
        self.censor_weights = {
            "numerator": self.fit_logistic_regression(self.data, numerator, censor_event),
            "denominator": self.fit_logistic_regression(self.data, denominator, censor_event)
        }
        with open(save_path, "wb") as f:
            pickle.dump(self.censor_weights, f)
    
    def calculate_weights(self):
        """Ensures that weights are properly estimated before proceeding."""
        if self.switch_weights is None or self.censor_weights is None:
            raise ValueError("Switch weights and/or censor weights have not been estimated. Run set_switch_weight_model and set_censor_weight_model first.")
        return self  # Allows method chaining
    
    def show_weight_models(self):
        """Displays the weight models."""
        print("Switch Weight Models:")
        if self.switch_weights:
            print(self.switch_weights["numerator"].summary())
        print("Censor Weight Models:")
        if self.censor_weights:
            print(self.censor_weights["numerator"].summary())

# Initialize trial sequences
data_censored = pd.read_csv("../data/data_censored.csv")
trial_pp = TrialSequence("PP", data_censored, "id", "period", "treatment", "outcome", "eligible")
trial_itt = TrialSequence("ITT", data_censored, "id", "period", "treatment", "outcome", "eligible")

# Define directories
trial_pp_dir = os.path.join(tempfile.gettempdir(), "trial_pp")
trial_itt_dir = os.path.join(tempfile.gettempdir(), "trial_itt")
os.makedirs(trial_pp_dir, exist_ok=True)
os.makedirs(trial_itt_dir, exist_ok=True)

# Fit weight models
switch_model_path = os.path.join(trial_pp_dir, "switch_models.pkl")
trial_pp.set_switch_weight_model(["age"], ["age", "x1", "x3"], switch_model_path)

censor_model_path = os.path.join(trial_pp_dir, "censor_models.pkl")
trial_pp.set_censor_weight_model("censored", ["x2"], ["x2", "x1"], censor_model_path)


In [81]:
# Calculate weights
trial_pp = trial_pp.calculate_weights()
trial_itt = trial_itt.calculate_weights()

ValueError: Switch weights and/or censor weights have not been estimated. Run set_switch_weight_model and set_censor_weight_model first.

In [83]:
# Show weight models
trial_itt.show_weight_models()
trial_pp.show_weight_models()

Switch Weight Models:
Censor Weight Models:
Switch Weight Models:
                           Logit Regression Results                           
Dep. Variable:               eligible   No. Observations:                  725
Model:                          Logit   Df Residuals:                      723
Method:                           MLE   Df Model:                            1
Date:                Sun, 09 Mar 2025   Pseudo R-squ.:                 0.09696
Time:                        16:39:57   Log-Likelihood:                -356.57
converged:                       True   LL-Null:                       -394.86
Covariance Type:            nonrobust   LLR p-value:                 2.120e-18
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.9822      0.383      5.180      0.000       1.232       2.732
age           -0.0692      0.008     -8.152      0.000      -0.08

## **5. Specify Outcome Model**

Now we can specify the outcome model. Here we can include adjustment terms for any variables in the dataset. The numerator terms from the stabilised weight models are automatically included in the outcome model formula.



In [84]:
# Apply outcome model to trial sequences
trial_pp.set_outcome_model()
trial_itt.set_outcome_model(adjustment_terms=["x2"])


AttributeError: 'TrialSequence' object has no attribute 'set_outcome_model'

## **6. Expand Trials**

### **6.1 Create Sequence of Trials Data**

## **7. Load or Sample from Expanded Data**

## **8. Fit Marginal Structural Model**

## **9. Inference**